<h1>OSM Data Wrangling </h1>
<hr>

<h2>Overview</h2>

In this project we will go over the data from the Open Street Map.

<h4>Table of Contents: </h4>

<a href="#a">1. File and Sample Size</a>

<a href="#b"> 2. Counting Users and Tags </a>

<a href="#c"> 3. Auditing Ways and Tags </a>

<a href="#d"> 4. Auditing and Fixing Street Names </a>

<a href="#e"> 5. Auditing and Fixing ZIP Codes </a>

<a href="#f"> 6. Turning XML to CSV </a>

<a href="#g"> 7. SQL Queries </a>

<a href="#h"> 8. Conclusion </a> 



In [2]:
#Here we are importing our dependencies
import xml.etree.ElementTree as ET
from pprint import pprint
import re
import os
from collections import defaultdict
import csv
import codecs
#import cerberus
import sqlite3


filename = 'san-francisco_california.osm'

OSM_PATH = 'san-francisco_california.osm'

osm_file = 'san-francisco_california.osm'



<h3><a id="a">File Size and Sample File</a></h3>

This file is almost 1.4 gigabytes. This is far too large for us to work with effectively. Working with a dataset this large will most likely crash the internet browser. We will create a smaller file by iterively going over the tags and adding them to the sf_sample.osm file. We can check our code on the sample file and run our code on the entire dataset when we are finished. 

In [3]:
size = os.path.getsize(filename)

print(size)

1397427031


In [4]:
sample_file = "sf_sample.osm"

k = 50

#Here we are iterating over the enitering dataset and looking for "start" and "end" events
def get_element(osm_file, tags=('node', 'way')):
    context = iter(ET.iterparse(osm_file, events=('start', 'end')))
    _, root = next(context)
    for event, elem in context:
        if event == 'end':
            yield elem
            root.clear()


with open(sample_file, 'wb') as output:
    output.write(b'<?xml version="1.0" encoding="UTF-8"?>\n')
    output.write(b'<osm>\n  ')

    for i, element in enumerate(get_element(osm_file)):
        if i % k == 0:
            output.write(ET.tostring(element, encoding='utf-8'))

    output.write(b'</osm>')

<h3><a id="b">Counting Users and Tags</a></h3>

First we go over the tags and see the different types and number of tags. There are far more low-level tags than high level tags which is good because that is what we were expecting. Next, we will look at how many people have contributed to the open street map project. We see that 1764 different people have contributed to the project. This is a surprisingly small number in my opinion because as you can see in the next cell we see that the total number of tags is a little over 17 million, this means that the average user contributed almost 10,000 tags! Finally, in the last cell we look at the sample file and calculate that it has 3% of the tags of our original file. 

In [9]:
#This code gives us a high-level overview of our file. We create a dictionary that will hold the tags, eays and nodes
#and count the parent and child elements for each

tags_dict = {}

def count_tags_total(filename):
    for elem in get_element(filename):
        if elem.tag in tags_dict:
            tags_dict[elem.tag] += 1
        elif elem.tag not in tags_dict:
            tags_dict[elem.tag] = 1
             
    return tags_dict

counted_tags_total = count_tags_total(filename)

pprint(counted_tags_total)
    

{'bounds': 1,
 'member': 54937,
 'nd': 7774045,
 'node': 6559055,
 'osm': 1,
 'relation': 6223,
 'tag': 2123923,
 'way': 807494}


In [16]:
def count_users(filename):
    unique_users = {}
    for elem in get_element(filename):
        for tag in elem.iter('way'):
            if tag.attrib['user'] not in unique_users:
                unique_users[tag.attrib['user']] = 1
            else:
                unique_users[tag.attrib['user']] += 1
    
    num_unique_users = len(unique_users)
    print (num_unique_users)
    
    return unique_users

count_users(filename);

1764


In [13]:
total_number_of_tags = tags_dict['nd'] + tags_dict['member'] + tags_dict['node'] + tags_dict['way'] + tags_dict['tag'] + tags_dict['relation']

print(total_number_of_tags)

total_users = 1764

average_tags_per_user = total_number_of_tags/total_users

print(average_tags_per_user)

17325677
9821.812358276644


In [20]:
tags_dict_sample = {}

def count_tags_sample(filename):
    for elem in get_element(filename):
        if elem.tag in tags_dict_sample:
            tags_dict_sample[elem.tag] += 1
        elif elem.tag not in tags_dict:
            tags_dict_sample[elem.tag] = 1
             
    return tags_dict_sample

counted_tags_sample = count_tags_sample(sample_file)

total_number_of_tags_sample = tags_dict_sample['nd'] + tags_dict_sample['member'] + tags_dict_sample['node'] + tags_dict_sample['way'] + tags_dict_sample['tag'] + tags_dict_sample['relation']


pprint(counted_tags_sample)

print(total_number_of_tags_sample)

percent_file_size = total_number_of_tags_sample/total_number_of_tags

print(percent_file_size)

{'bounds': 1,
 'member': 1825,
 'nd': 311466,
 'node': 131127,
 'osm': 1,
 'relation': 119,
 'tag': 85163,
 'way': 16286}
545986
0.03151311201288123


<h3><a id="c">Auditing Ways and Tags</a></h3>

We are looking closer at the ways tags here. This gives us a very basic overview of our ways tags. Going a little deeper in the next cell, we can clearly see familiar text such as street names and house numbers. Lets keep going further in our auditing process. There is a lot of information here stored in dictionaries. 

In [8]:
#A counter of 5 is used to keep the code clean and concise
def audit_ways(filename):
    counter = 0
    ways = []
    for elem in get_element(filename):
        if counter < 5:
            for tag in elem.iter('way'):
                    attributes = {
                        "id": tag.attrib['id'],
                        "version": tag.attrib['version'],
                        'timestamp': tag.attrib['timestamp'],
                        'changeset': tag.attrib['changeset'],
                        'uid': tag.attrib['uid'],
                        'user': tag.attrib['user']
                    }
                    ways.append(tag)
                    ways.append(attributes)
                    counter += 1
    return ways

sf_attributes = audit_ways(sample_file)

pprint(sf_attributes)

[<Element 'way' at 0x263a278>,
 {'changeset': '36649660',
  'id': '4970189',
  'timestamp': '2016-01-18T06:57:39Z',
  'uid': '296869',
  'user': 'Zian Choy',
  'version': '28'},
 <Element 'way' at 0x5c04390>,
 {'changeset': '5377692',
  'id': '5149040',
  'timestamp': '2010-08-02T07:27:12Z',
  'uid': '24452',
  'user': 'Speight',
  'version': '4'},
 <Element 'way' at 0x5c046d8>,
 {'changeset': '41210984',
  'id': '5149917',
  'timestamp': '2016-08-03T10:09:20Z',
  'uid': '933797',
  'user': 'oba510',
  'version': '8'},
 <Element 'way' at 0x5a2ca20>,
 {'changeset': '16977801',
  'id': '6319820',
  'timestamp': '2013-07-16T17:52:08Z',
  'uid': '153669',
  'user': 'dchiles',
  'version': '9'},
 <Element 'way' at 0x5a7b048>,
 {'changeset': '445579',
  'id': '6323786',
  'timestamp': '2007-09-17T03:40:46Z',
  'uid': '7168',
  'user': 'DaveHansenTiger',
  'version': '1'}]


In [6]:
#We are now going a little bit deeper in our code and printing the attributes of each of the elements


def audit_tags(filename):
    counter = 0
    attributes = defaultdict(set)
    for elem in get_element(filename):
        if counter < 100:
            for tag in elem.iter('tag'):
                attributes[tag.attrib['k']].add(tag.attrib['v'])
                counter += 1

    return attributes

sf_attributes = audit_tags(sample_file)
print(len(sf_attributes))

pprint(sf_attributes)


15
defaultdict(<type 'set'>, {'toilets': set(['yes']), 'fixme': set(['Needs Ground Survey. Road closed to through traffic near here']), 'exit_to': set(['Winton Avenue', '11th Street;14th Street']), 'barrier': set(['gate', 'lift_gate']), 'power': set(['tower']), 'name:zh': set([u'\u65e7\u91d1\u5c71']), 'created_by': set(['JOSM']), 'source': set(['http://www.dot.ca.gov/hq/traffops/signtech/calnexus/pdf/980.pdf', 'http://www.dot.ca.gov/hq/traffops/signtech/calnexus/pdf/eighteightynorth.pdf;http://www.dot.ca.gov/hq/traffops/signtech/calnexus/pdf/eighteightysouth.pdf', 'PGS', 'local_knowledge']), 'ref:right': set(['17A']), 'foot': set(['yes']), 'crossing': set(['uncontrolled']), 'ref': set(['1A', '46', '25A', '27', '23']), 'exit_to:right': set(['2nd Street']), 'highway': set(['turning_circle', 'crossing', 'stop', 'traffic_signals', 'motorway_junction']), 'noref': set(['yes'])})


<h3><a id="d"> Audit and Fixing Street Names</a> </h3> 

Here we are taking a deeper look at the data, more specifically the street names. We are auditing the street names and seeing if they match our expected spelling. You can see below that we have a several street names that are spelled incorrectly. This dataset is extremely large, so for now we will only focus on trying to fix the incorrect spelling of the street name suffix. From our audit of our sample file, we see that we have 3 examples of this issue: 'California Dr', 'Magnolia Ave' and 'Sawyer Camp Trail & Hillcrest Blvd'. We implement a function that loops through the incorrect names and removes the incorrect suffix, then appends the correct suffix to the street name. As you can see we successfully fixed these street names.

In [6]:
expected = ['Street', 'Avenue', 'Drive', 
            'Road', 'Boulevard', 'Court',
            'Place', 'Highway', 'Way', 
            'Alley', 'Plaza', 'Circle',
            'Park', 'Lane', 'Parkway', 
            'Path', 'Road', 'Terrace',
            'Center', 'Square', "North", 
            "East", "West", "South"]

def audit_street_names(filename):
    unexpected_street_names = defaultdict(set)
    for elem in get_element(filename):
        for tag in elem.iter('tag'):
            if tag.attrib['k'] == 'addr:street':
                #Declaring the street name and the suffix and adding it to the dictionary if it is not in the expected array
                street_name = tag.attrib['v'].split()
                street_name_suffix = street_name[-1]
                for word in street_name:
                     if street_name_suffix not in expected:
                        unexpected_street_names[tag.attrib['k']].add(tag.attrib['v'])
                        
    return unexpected_street_names

audited_street_names = audit_street_names(sample_file)

pprint(audited_street_names)

defaultdict(<type 'set'>, {'addr:street': set(['Bridgeway', 'Sawyer Camp Trail & Hillcrest Blvd', 'The Alameda', 'Alameda', '12180142', 'Magnolia Ave', 'Rose Walk', 'Avenue H', 'Alameda De Las', 'California', 'Post', 'Garvin Avenie', 'Pacific Marina', 'San Clemente', 'El Camino Real', 'Wellesley Cres', 'Wildwood Gardens', 'Santa Cruz avenue', 'West El Camino Real', 'Woodside Plz', 'California Dr', 'Mission', 'San Francisco Bicycle Route 2', 'Broadway', 'Fort Mason'])})


In [24]:
#This dictionary contains the incorrect abbreviated spelling as the key and the correct spelling as the value
mapping = { 
            "St": "Street",
            "St.": "Street",
            "STREET": "Street",
            "Ave": "Avenue",
            "Ave.": "Avenue",
            "Dr.": "Drive",
            "Dr": "Drive",
            "Rd": "Road",
            "Rd.": "Road",
            "Blvd": "Boulevard",
            "Blvd.": "Boulevard",
            "Ehs": "EHS",
            "Trl": "Trail",
            "Cir": "Circle",
            "Cir.": "Circle",
            "Ct": "Court",
            "Ct.": "Court",
            "Crt": "Court",
            "Crt.": "Court",
            "By-pass": "Bypass",
            "N.": "North",
            "N": "North",
            "E.": "East",
            "E": "East",
            "S.": "South",
            "S": "South",
            "W.": "West",
            "W": "West"
          }


expected = ['Street', 'Avenue', 'Drive', 
            'Road', 'Boulevard', 'Court',
            'Place', 'Highway', 'Way', 
            'Alley', 'Plaza', 'Circle',
            'Park', 'Lane', 'Parkway', 
            'Path', 'Road', 'Terrace',
            'Center', 'Square', "North", 
            "East", "West", "South"]


"""
    In this fuction we remove the incorrect suffix 
    and append the correct spelling. Then we append the correct street name to the array.
"""
def fix_street_names(filename):
    counter = 0
    fixed_street_names_Arr = []
    for elem in get_element(filename):
        for tag in elem.iter('tag'):
            if tag.attrib['k'] == 'addr:street':
                street_name = tag.attrib['v'].split()
                street_name_suffix = street_name[-1]
                for word in street_name:
                     if street_name_suffix not in expected and counter < 100:
                        if street_name_suffix in mapping:
                            correct_spelling = mapping.get(street_name_suffix)
                            street_name_suffix = correct_spelling
                            street_name.pop()
                            street_name.append(street_name_suffix)
                            fixed_street_names_Arr.append(street_name)
                            counter += 1
    
    fixed_names = [' '.join(name) for name in fixed_street_names_Arr]
    return fixed_names

fixed_street_names = fix_street_names(sample_file)

pprint(fixed_street_names)
    

['California Drive',
 'Magnolia Avenue',
 'Sawyer Camp Trail & Hillcrest Boulevard']


<h3><a id='e'>Auditing and Fixing ZIP codes</a></h3>

We can also audit ZIP codes in a similar way that we audited the street names. In the next cell, we are running code to see if there are any ZIP Codes that dont match the expected 5 number format. We see that there are 5 examples of this. Lets fix that in the next cell. We can implement a simple solution to fix our ZIP codes. We simply loop over the incoorect formats and convert them to the 5 number format. As you can see the ZIP codes from the last cell are in the correct formatting now.

In [23]:
def audit_zipcode(filename):
    ways = []
    for elem in get_element(filename):
        if elem.tag == 'way':
            for child in elem.iter('tag'):
                if child.attrib['k'] == "addr:postcode":
                    if len(child.attrib['v']) != 5:
                        ways.append(child.attrib['v'])

    return ways
audited_zipcode = audit_zipcode(sample_file)

pprint(audited_zipcode)

['94002-3585', 'CA 94544', '94301-2019', '94117-9991', '94118-1316']


In [30]:
def fix_zip_codes(filename):
    zip_codes = []
    for elem in get_element(filename):
        if elem.tag == 'way':
            for child in elem.iter('tag'):
                if child.attrib['k'] == "addr:postcode":
                    zip_code = child.attrib['v']
                    if len(zip_code) == 10:
                        zip_code = zip_code[:-5]
                        zip_codes.append(zip_code)
                    elif len(zip_code) == 8:
                        zip_code = zip_code[3:]
                        zip_codes.append(zip_code)
                    elif len(zip_code) != 5:
                        zip_code.append(zip_codes)
    return zip_codes
                        
fixed_zip_code = fix_zip_codes(sample_file)

pprint(fixed_zip_code)

['94002', '94544', '94301', '94117', '94118']


<h3><a id="f"> Turning XML to CSV</a></h3>

Here we will start to convert our XML file to CSV to use with SQL queries. 

In [26]:

NODE_FIELDS = ['id', 'lat', 'lon', 'user', 'uid', 'version', 'changeset', 'timestamp']
NODE_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_FIELDS = ['id', 'user', 'uid', 'version', 'changeset', 'timestamp']
WAY_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_NODES_FIELDS = ['id', 'node_id', 'position']

NODES_PATH = "nodes.csv"
NODE_TAGS_PATH = "nodes_tags.csv"
WAYS_PATH = "ways.csv"
WAY_NODES_PATH = "ways_nodes.csv"
WAY_TAGS_PATH = "ways_tags.csv"

lower = re.compile(r'^([a-z]|_)*$')
lower_colon = re.compile(r'^([a-z]|_)*:([a-z]|_)*$')
PROBLEMCHARS = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')

In [27]:
def shape_element(elem):

    ways = {
        'way': {},
        'way_nodes': [],
        'way_tags': []
    }
    nodes = {
        'node': {},
        'node_tags': []
    }
    
    if elem.tag == 'way':
        way_attributes = {
            'id': elem.attrib['id'],
            'version': elem.attrib['version'],
            'timestamp': elem.attrib['timestamp'],
            'changeset': elem.attrib['changeset'],
            'uid': elem.attrib['uid'],
            'user': elem.attrib['user']
                 }
        ways['way'].update(way_attributes)


        node_counter = 0
        for child in elem.iter('nd'):
            way_node_attributes = {
                'id': elem.attrib['id'],
                'node_id': child.attrib['ref'],
                'position': node_counter
                  }
            ways['way_nodes'].append(way_node_attributes)
            node_counter += 1
                
        for child in elem.iter('tag'):
            cid = elem.attrib['id']
            key = child.attrib['k']
            if ':' in key:
                pos = key.find(':')
                key = key[pos+1:]
                ktype = key[:pos]
            else:
                ktype = 'regular'

            tag_attributes = {
                'id': elem.attrib['id'],
                "key": child.attrib['k'],
                'type': ktype,
                'value': child.attrib['v']
                    }
            ways['way_tags'].append(tag_attributes)

    if elem.tag == 'node':
        node_attributes =  {'id': elem.attrib['id'],
                      'user': elem.attrib['user'],
                      'uid': elem.attrib['uid'],
                      'version': elem.attrib['version'],
                      'lat': elem.attrib['lat'],
                      'lon': elem.attrib['lon'],
                      'timestamp': elem.attrib['timestamp'],
                      'changeset': elem.attrib['changeset']
                    }

        nodes['node'].update(node_attributes)
        for child in elem.iter('tag'):
            cid = elem.attrib['id']
            key = child.attrib['k']
            if ':' in key:
                pos = key.find(':')
                key = key[pos+1:]
                ktype = key[:pos]
            else:
                ktype = 'regular'


            node_tag_attrib = {
                    'id':cid,
                    'key': key,
                    'value': child.attrib['v'],
                    'type': ktype
                }
            nodes['node_tags'].append(node_tag_attrib)


    if elem.tag =='node':
        return nodes
    elif elem.tag == 'way':
        return ways




In [28]:
Helper Functions

def validate_element(element, validator, schema=SCHEMA):
    """Raise ValidationError if element does not match schema"""
    if validator.validate(element, schema) is not True:
        field, errors = next(validator.errors.iteritems())
        message_string = "\nElement of type '{0}' has the following errors:\n{1}"
        error_string = pprint.pformat(errors)
        
        raise Exception(message_string.format(field, error_string))


class UnicodeDictWriter(csv.DictWriter, object):
    """Extend csv.DictWriter to handle Unicode input"""

    def writerow(self, row):
        super(UnicodeDictWriter, self).writerow({
            k: (v.encode('utf-8') if isinstance(v, unicode) else v) for k, v in row.items()
        })

    def writerows(self, rows):
        for row in rows:
            self.writerow(row)


In [29]:
#Main Function

def process_map(file_in, validate):
    """Iteratively process each XML element and write to csv(s)"""

    with codecs.open(NODES_PATH, 'w') as nodes_file, \
         codecs.open(NODE_TAGS_PATH, 'w') as nodes_tags_file, \
         codecs.open(WAYS_PATH, 'w') as ways_file, \
         codecs.open(WAY_NODES_PATH, 'w') as way_nodes_file, \
         codecs.open(WAY_TAGS_PATH, 'w') as way_tags_file:

        nodes_writer = UnicodeDictWriter(nodes_file, NODE_FIELDS)
        node_tags_writer = UnicodeDictWriter(nodes_tags_file, NODE_TAGS_FIELDS)
        ways_writer = UnicodeDictWriter(ways_file, WAY_FIELDS)
        way_nodes_writer = UnicodeDictWriter(way_nodes_file, WAY_NODES_FIELDS)
        way_tags_writer = UnicodeDictWriter(way_tags_file, WAY_TAGS_FIELDS)

        nodes_writer.writeheader()
        node_tags_writer.writeheader()
        ways_writer.writeheader()
        way_nodes_writer.writeheader()
        way_tags_writer.writeheader()
        
#         validator = cerberus.Validator()

        for elem in get_element(file_in, tags=('node', 'way')):
            el = shape_element(elem)
            if el:
                if validate is True:
                    validate_element(el, validator)

                if elem.tag == 'node':
                    nodes_writer.writerow(el['node'])
                    node_tags_writer.writerows(el['node_tags'])
                elif elem.tag == 'way':
                    ways_writer.writerow(el['way'])
                    way_nodes_writer.writerows(el['way_nodes'])
                    way_tags_writer.writerows(el['way_tags'])



process_map(sample_file, validate=False);



In [16]:
SCHEMA = """CREATE TABLE nodes ( 
    id INTEGER PRIMARY KEY NOT NULL,
    lat REAL,
    lon REAL,
    user TEXT,
    uid INTEGER,
    version INTEGER,
    changeset INTEGER,
    timestamp TEXT
);

CREATE TABLE nodes_tags (
    id INTEGER,
    key TEXT,
    value TEXT,
    type TEXT,
    FOREIGN KEY (id) REFERENCES nodes(id)
);

CREATE TABLE ways (
    id INTEGER PRIMARY KEY NOT NULL,
    user TEXT,
    uid INTEGER,
    version TEXT,
    changeset INTEGER,
    timestamp TEXT
);

CREATE TABLE ways_tags (
    id INTEGER NOT NULL,
    key TEXT NOT NULL,
    value TEXT NOT NULL,
    type TEXT,
    FOREIGN KEY (id) REFERENCES ways(id)
);

CREATE TABLE ways_nodes (
    id INTEGER NOT NULL,
    node_id INTEGER NOT NULL,
    position INTEGER NOT NULL,
    FOREIGN KEY (id) REFERENCES ways(id),
    FOREIGN KEY (node_id) REFERENCES nodes(id)
);"""

In [5]:
#In this cell we will create all our tables for use in SQL queries


db_file = 'sf_osm.db'

conn = sqlite3.connect(db_file)



"""
    Nodes
    
"""

cur = conn.cursor()

cur.execute('''DROP TABLE IF EXISTS nodes''')

conn.commit()


cur = conn.cursor()

cur.execute(
    """CREATE TABLE nodes ( 
    id INTEGER PRIMARY KEY NOT NULL,
    lat REAL,
    lon REAL,
    user TEXT,
    uid INTEGER,
    version INTEGER,
    changeset INTEGER,
    timestamp TEXT
);
""")

conn.commit()


with open('nodes.csv','r') as fin:
    dr = csv.DictReader(fin) # comma is default delimiter
    to_db = [(i['id'], i['lat'].decode("utf-8"), i['lon'].decode("utf-8"), i['user'].decode("utf-8"), i['uid'].decode("utf-8"), i['version'].decode("utf-8"), i['changeset'].decode("utf-8"), i['timestamp'].decode("utf-8")) for i in dr]


cur.executemany("INSERT INTO nodes(id, lat, lon, user, uid, version, changeset, timestamp) VALUES (?, ?, ?, ?, ?, ?, ?, ?);", to_db)


conn.commit()



"""
        Nodes_Tags

"""

cur = conn.cursor()

cur.execute('''DROP TABLE IF EXISTS nodes_tags''')

conn.commit()



cur = conn.cursor()

cur.execute('''
    CREATE TABLE nodes_tags(id INTEGER, key TEXT, value TEXT,type TEXT)
''')

conn.commit()


with open('nodes_tags.csv','r') as fin:
    dr = csv.DictReader(fin) # comma is default delimiter
    to_db = [(i['id'],i['key'].decode("utf-8"),i['value'].decode("utf-8"),i['type']) for i in dr]


cur.executemany("INSERT INTO nodes_tags(id, key, value, type) VALUES (?, ?, ?, ?);", to_db)


conn.commit()




"""
        Ways

"""

cur = conn.cursor()

cur.execute('''DROP TABLE IF EXISTS ways''')

conn.commit()


cur = conn.cursor()

cur.execute('''
    CREATE TABLE ways (
    id INTEGER PRIMARY KEY NOT NULL,
    user TEXT,
    uid INTEGER,
    version TEXT,
    changeset INTEGER,
    timestamp TEXT
);
''')

conn.commit()


with open('ways.csv','r') as fin:
    dr = csv.DictReader(fin) # comma is default delimiter
    to_db = [(i['id'], i['user'].decode("utf-8"), i['uid'].decode("utf-8"), i['version'].decode("utf-8"), i['changeset'].decode("utf8"), i['timestamp'].decode("utf-8")) for i in dr]


cur.executemany("INSERT INTO ways(id, user, uid, version, changeset, uid) VALUES (?, ?, ?, ?, ?, ?);", to_db)


conn.commit()

"""
        Ways_Nodes

"""


cur = conn.cursor()

cur.execute('''DROP TABLE IF EXISTS ways_nodes''')

conn.commit()


cur = conn.cursor()

cur.execute('''
    CREATE TABLE ways_nodes (
    id INTEGER NOT NULL,
    node_id INTEGER NOT NULL,
    position INTEGER NOT NULL,
    FOREIGN KEY (id) REFERENCES ways(id),
    FOREIGN KEY (node_id) REFERENCES nodes(id)
);
''')

conn.commit()


with open('ways_nodes.csv','r') as fin:
    dr = csv.DictReader(fin) # comma is default delimiter
    to_db = [(i['id'], i['node_id'].decode("utf-8"), i['position'].decode("utf-8")) for i in dr]


cur.executemany("INSERT INTO ways_nodes(id, node_id, position) VALUES (?, ?, ?);", to_db)


conn.commit()

"""
        Ways_Tags

"""

cur = conn.cursor()

cur.execute('''DROP TABLE IF EXISTS ways_tags''')

conn.commit()


cur = conn.cursor()

cur.execute('''
    CREATE TABLE ways_tags (
    id INTEGER NOT NULL,
    key TEXT NOT NULL,
    value TEXT NOT NULL,
    type TEXT,
    FOREIGN KEY (id) REFERENCES ways(id)
);
''')

conn.commit()


with open('ways_tags.csv','r') as fin:
    dr = csv.DictReader(fin) # comma is default delimiter
    to_db = [(i['id'], i['key'].decode("utf-8"), i['value'].decode("utf-8"), i['type'].decode('utf-8')) for i in dr]


cur.executemany("INSERT INTO ways_tags(id, key, value, type) VALUES (?, ?, ?, ?);", to_db)


conn.commit()

    


<h3> <a id='g'> SQL Queries </a></h3>

We will begin with very basic SQL Queries, then go deeper. 

In [30]:
#Here we see the first 10 rows from each of our columns
cur.execute('SELECT * FROM nodes_tags LIMIT 10')
all_rows = cur.fetchall()
print('nodes_tags:')
pprint(all_rows)

nodes_tags
[(30372583, u'ref', u'1A', u'regular'),
 (30372583,
  u'source',
  u'http://www.dot.ca.gov/hq/traffops/signtech/calnexus/pdf/980.pdf',
  u'regular'),
 (30372583, u'exit_to', u'11th Street;14th Street', u'regular'),
 (30372583, u'highway', u'motorway_junction', u'regular'),
 (31817516, u'source', u'PGS', u'regular'),
 (31827991, u'source', u'PGS', u'regular'),
 (31849484, u'source', u'PGS', u'regular'),
 (31857880, u'source', u'PGS', u'regular'),
 (35706322, u'created_by', u'JOSM', u'regular'),
 (35706444, u'created_by', u'JOSM', u'regular')]


In [31]:
cur.execute('SELECT * FROM ways LIMIT 10')
all_rows = cur.fetchall()
print("ways:")
pprint(all_rows)

ways:
[(4970189, u'Zian Choy', 296869, u'28', 36649660, None),
 (5149040, u'Speight', 24452, u'4', 5377692, None),
 (5149917, u'oba510', 933797, u'8', 41210984, None),
 (6319820, u'dchiles', 153669, u'9', 16977801, None),
 (6323786, u'DaveHansenTiger', 7168, u'1', 445579, None),
 (6325632, u'DanHomerick', 160138, u'2', 11085608, None),
 (6326935, u'balrog-kun', 20587, u'2', 4244079, None),
 (6326959, u'balrog-kun', 20587, u'5', 4244079, None),
 (6327093, u'balrog-kun', 20587, u'2', 4244079, None),
 (6327122, u'KindredCoda', 14293, u'10', 10996498, None)]


In [32]:
cur.execute('SELECT * FROM ways_nodes LIMIT 10')
all_rows = cur.fetchall()
print("ways_nodes:")
pprint(all_rows)

ways_tags:
[(4970189, 32926684, 0),
 (4970189, 272607326, 1),
 (4970189, 1635648542, 2),
 (4970189, 541332511, 3),
 (4970189, 257013067, 4),
 (4970189, 541332512, 5),
 (4970189, 541332513, 6),
 (4970189, 29891951, 7),
 (4970189, 1635648546, 8),
 (4970189, 750523210, 9)]


In [22]:
cur.execute('SELECT * FROM ways_tags LIMIT 10')
all_rows = cur.fetchall()
print('ways_tags:')
pprint(all_rows)

ways_tags:
[(4970189, u'NHS', u'yes', u'regular'),
 (4970189, u'hgv', u'designated', u'regular'),
 (4970189, u'ref', u'CA 92', u'regular'),
 (4970189, u'name', u'J Arthur Younger Freeway', u'regular'),
 (4970189, u'lanes', u'2', u'regular'),
 (4970189, u'oneway', u'yes', u'regular'),
 (4970189, u'bicycle', u'no', u'regular'),
 (4970189, u'highway', u'motorway', u'regular'),
 (4970189, u'maxspeed', u'55 mph', u'regular'),
 (4970189, u'source:maxspeed', u'sign', u'maxspe')]


In [42]:
cur.execute('SELECT * FROM nodes LIMIT 5')
all_rows = cur.fetchall()
print('nodes:')
pprint(all_rows)

nodes:
[(29891972,
  37.6714631,
  -122.3912181,
  u'KindredCoda',
  14293,
  6,
  15682910,
  u'2013-04-10T20:56:34Z'),
 (30029500,
  37.6890322,
  -122.1078783,
  u'bill42',
  482916,
  20,
  10936318,
  u'2012-03-10T20:18:41Z'),
 (30033745,
  37.8562329,
  -122.4950289,
  u'Speight',
  24452,
  7,
  74256,
  u'2009-04-02T07:39:02Z'),
 (30362588,
  37.8366965,
  -122.2958607,
  u'Speight',
  24452,
  8,
  5487612,
  u'2010-08-14T06:23:43Z'),
 (30364114,
  37.8228807,
  -122.3011238,
  u'dannykath',
  2226712,
  5,
  36818926,
  u'2016-01-26T15:09:31Z')]


In [6]:
#Here we see that the most nodes a way tag has is 722 nodes! 
cur.execute("""SELECT position, id
               FROM ways_nodes
               ORDER BY position DESC
               LIMIT 1
               """)

all_rows = cur.fetchall()
pprint(all_rows)

[(722, 192565555)]


In [11]:
#Here we have the top contributer of the ways tags with 16286 tags
cur.execute(""" SELECT user, COUNT(*) as num
                FROM  ways
                ORDER BY num DESC
                LIMIT 10;
               """)

all_rows = cur.fetchall()
pprint(all_rows)

[(u'AndrewSnow', 16286)]


In [13]:
#Here we have the top 10 amenities
cur.execute(""" SELECT value, COUNT(*) as num
                FROM nodes_tags
                WHERE key='amenity'
                GROUP BY value
                ORDER BY num DESC
                LIMIT 10;
               """)

all_rows = cur.fetchall()
pprint(all_rows)

[(u'restaurant', 63),
 (u'bench', 25),
 (u'cafe', 22),
 (u'drinking_water', 14),
 (u'fast_food', 13),
 (u'toilets', 13),
 (u'post_box', 12),
 (u'place_of_worship', 11),
 (u'school', 11),
 (u'bank', 8)]


In [21]:
#Here we see that Irving Street is the most documented street in the nodes_tags
cur.execute(""" SELECT value, COUNT(*) as num
                FROM nodes_tags
                WHERE key='street'
                GROUP BY value
                ORDER BY num DESC
                LIMIT 10;
               """)

all_rows = cur.fetchall()
pprint(all_rows)

[(u'Irving Street', 10),
 (u'Church Street', 8),
 (u'Divisadero Street', 8),
 (u'Martin Luther King Jr Way', 7),
 (u'Page Street', 7),
 (u'Woodside Road', 7),
 (u'9th Avenue', 5),
 (u'Broadway', 5),
 (u'Dolores Street', 5),
 (u'Judah Street', 5)]


<h3><a id='h'>Conclusion</a></h3>

One problem I encountered was when I was looking through the tags after running my audit_tags function. The problem was with dead URLs that were cited as sources for certain tags. I noticed the dead URLs were usually government websites such as:  http://www.dot.ca.gov/hq/traffops/trucks/truckmap/ and ftp2.census.gov/geo/tiger/TIGER2013/ROADS/tl_2013_06081_roads. Some government URLs did actually work however, such as http://waterdata.usgs.gov/nwis/. The other URLs that were from the private sector generally worked such as 'https://www.propertyshark.com/mason/ca/Contra-Costa-County/Maps'. Without legitimate sources, people can just make up incorrect or inaccurate information, so this is a real issue. One way to correct this would be to periodically check if the URLs are valid. There might be a way to accomplish this programatically, but that is beyond my skill level. Another way to solve this would be government websites working with private websites. Private websites that sell information about real estate market trends, such as the previously mentioned propertyshark.com, have a real incentive to have correct and up to date information. Private websites might be willing to work with government websites because it will be a "win win" situation. The private website will have access to the government websites map information and resources and the government website will have access to the private websites information and resources. Having detailed map information about metropolitan areas is difficult and time-comsuming. By combining and pooling their resources, government and private websites will both be better off and have 1 very detailed map, instead of 2 not-very detailed maps. However, There are a lot obstacles to this. Government and Private orginizations generally do not work together for obvious reasons. There are also financial and planning issues that have to be accounted for.






<hr>
<h4>Resources:</h4>
<hr>

https://discussions.udacity.com/t/p3-openstreetmap-overview/172045/2<br>
https://discussions.udacity.com/t/final-project-need-help-getting-started/181525/60<br>
https://discussions.udacity.com/t/count-tags-function-implementation/264133/2<br>
https://discussions.udacity.com/t/system-ram-goes-to-99-while-wrangling/182980/7<br>
https://wiki.openstreetmap.org/wiki/Elements<br>
http://effbot.org/zone/element.htm<br>
https://stackoverflow.com/questions/15881497/retrieving-xml-attribute-values-using-python-iterparse<br>
https://discussions.udacity.com/t/elements-and-attributes/267615<br>
https://gist.github.com/carlward/54ec1c91b62a5f911c42#file-sample_project-md<br>
http://stackoverflow.com/questions/3095434/inserting-newlines-in-xml-file-generated-via-xml-etree-elementtree-in-python<br>
https://stackoverflow.com/questions/15181867/understanding-the-set-function<br>
https://discussions.udacity.com/t/printing-tags-for-keys-and-values/273966/3<br>
https://stackoverflow.com/questions/5518435/python-fastest-way-to-create-a-list-of-n-lists<br>
https://discussions.udacity.com/t/creating-a-list-of-lists-of-ways-tags/274031<br>
https://discussions.udacity.com/t/help-with-count-users-function/274167<br>
https://discussions.udacity.com/t/help-with-audit-street-names-function/274268<br>
https://discussions.udacity.com/t/feedback-on-audit-street-names-method/275561<br>
https://discussions.udacity.com/t/action-plan-for-p3/231324<br>
https://github.com/brownan/SQLite-tutorial/blob/master/SQLite_Tutorial.ipynb<br>
https://discussions.udacity.com/t/filenotfounderror-errno-2-no-such-file-or-directory-nodes-tags-csv/287485/14<br>
https://discussions.udacity.com/t/unicodeencodeerror-charmap-codec-cant-encode-characters-in-position-32-37-character-maps-to-undefined/288697<br>

